In [1]:
from models import custom_RF as rf
from load_dataset import custom_lakedepth as ld

In [2]:
data = ld.LakeDepth(42)
cv_train, cv_test, l_train, l_test = data.split(0.20)

 - from DATA: reading csv into GPU memory
 - from DATA: done reading csv into GPU memory
 - from DATA: dropped column: FID
 - from DATA: dropped column: Date


In [3]:
hyperparameters = {'N_ESTIMATORS' : 1150,
					   'SPLIT_ALGO' : 1,
					   'SPLIT_CRITERION' : 2,
					   'BOOTSTRAP' : True,
					   'BOOTSTRAP_FEATURES' : False,
					   'ROWS_SAMPLE' : 1.0,
					   'MAX_DEPTH' : 110,
					   'MAX_LEAVES' : -1,
					   'MAX_FEATURES' : 'sqrt',
					   'N_BINS' : 6,
					   'MIN_ROWS_PER_NODE' : 2,
					   'MIN_IMPURITY_DECREASE' : 0.0,
					   'ACCURACY_METRIC' : 'mean_ae', # 'mse' #'r2' # 'median_aw' # 
					   'QUANTILEPT' : False,
					   'SEED' :  42,
					   'VERBOSE' : False
					   }
from pprint import pprint
pprint(hyperparameters)

{'ACCURACY_METRIC': 'mean_ae',
 'BOOTSTRAP': True,
 'BOOTSTRAP_FEATURES': False,
 'MAX_DEPTH': 110,
 'MAX_FEATURES': 'sqrt',
 'MAX_LEAVES': -1,
 'MIN_IMPURITY_DECREASE': 0.0,
 'MIN_ROWS_PER_NODE': 2,
 'N_BINS': 6,
 'N_ESTIMATORS': 1150,
 'QUANTILEPT': False,
 'ROWS_SAMPLE': 1.0,
 'SEED': 42,
 'SPLIT_ALGO': 1,
 'SPLIT_CRITERION': 2,
 'VERBOSE': False}


In [4]:
rf_model1 = rf.cuRF(hyperparameters)

/att/gpfsfs/briskfs01/ppl/cssprad1/projects/rapids_rf_lake_depth/models/custom_RF.py:155: UserWarning: For reproducible results in Random Forest Classifier or for almost reproducible results in Random Forest Regressor, n_streams==1 is recommended. If n_streams is > 1, results may vary due to stream/thread timing differences, even when random_state is set
  verbose=param['VERBOSE'])


In [5]:
import time

st = time.time()
rf_model1.train(cv_train, l_train)
et = time.time()
print("Time to train (sec)", et-st)

Time to train (sec) 8.926137924194336


In [6]:
rf_model1.get_metrics(cv_test, l_test)

The history saving thread hit an unexpected error (OperationalError('database is locked')).History will not be written to the database.
Scores ------
 MAE:  0.47657734
  r2:  0.7557918428754148
 MSE:  0.7578933


(0.47657734, 0.7557918428754148, 0.7578933)

### Isolate the  b8 spectral band

We're gonna train without it and get the r2 score

In [7]:
data.covariates = data.covariates.drop(['b32_LC8_07'], axis=1)
cv_train1, cv_test1, l_train1, l_test1 = data.split(0.20)

In [8]:
cv_train1.head(5)

,b1_LC8_075,b2_LC8_075,b3_LC8_075,b4_LC8_075,b5_LC8_075,b6_LC8_075,b7_LC8_075,b8_LC8_075,b9_LC8_075,b10_LC8_07,...,b25_LC8_07,b26_LC8_07,b27_LC8_07,b28_LC8_07,b29_LC8_07,b30_LC8_07,b31_LC8_07,b33_LC8_07,b34_LC8_07,b35_LC8_07
1913,132.0,202.0,97.0,38.0,17.0,15.0,653.0,1361.0,3474.0,7765.0,...,2235.0,2533.0,129.0,84.0,175.0,447.0,114.0,155.0,395.0,882.0
8926,151.0,242.0,208.0,328.0,299.0,216.0,624.0,726.0,460.0,505.0,...,1097.0,1519.0,1980.0,1236.0,1438.0,912.0,1430.0,1038.0,659.0,722.0
14706,596.0,592.0,589.0,892.0,834.0,615.0,1007.0,1012.0,668.0,715.0,...,1070.0,1450.0,1399.0,1409.0,1416.0,935.0,1032.0,1044.0,689.0,737.0
3503,212.0,350.0,172.0,73.0,42.0,33.0,606.0,1233.0,2904.0,5048.0,...,1738.0,2212.0,198.0,120.0,244.0,575.0,156.0,192.0,452.0,786.0
7744,1219.0,1226.0,1146.0,913.0,350.0,289.0,994.0,1064.0,1335.0,3483.0,...,2609.0,3159.0,287.0,285.0,305.0,383.0,237.0,252.0,317.0,826.0


In [9]:
rf_model2 = rf.cuRF(hyperparameters)

/att/gpfsfs/briskfs01/ppl/cssprad1/projects/rapids_rf_lake_depth/models/custom_RF.py:155: UserWarning: For reproducible results in Random Forest Classifier or for almost reproducible results in Random Forest Regressor, n_streams==1 is recommended. If n_streams is > 1, results may vary due to stream/thread timing differences, even when random_state is set
  verbose=param['VERBOSE'])


In [10]:
rf_model2.train(cv_train1, l_train1)

In [11]:
r2s_list = []
_, r2_2, _ = rf_model2.get_metrics(cv_test1, l_test1)
r2s_list.append(r2_2)

Scores ------
 MAE:  0.49186188
  r2:  0.7403002871086541
 MSE:  0.8067346


In [12]:
rf_model3 = rf.cuRF(hyperparameters)
cv_3, cv_t3, l_3, l_t3 = data.split(0.20)
rf_model3.train(cv_3, l_3)
_, r2_3, _ = rf_model3.get_metrics(cv_t3, l_t3)
r2s_list.append(r2_3)

/att/gpfsfs/briskfs01/ppl/cssprad1/projects/rapids_rf_lake_depth/models/custom_RF.py:155: UserWarning: For reproducible results in Random Forest Classifier or for almost reproducible results in Random Forest Regressor, n_streams==1 is recommended. If n_streams is > 1, results may vary due to stream/thread timing differences, even when random_state is set
  verbose=param['VERBOSE'])


Scores ------
 MAE:  0.5017001
  r2:  0.7731209960404318
 MSE:  0.875981


In [13]:
rf_model4 = rf.cuRF(hyperparameters)
cv_4, cv_t4, l_4, l_t4 = data.split(0.20)
rf_model4.train(cv_4, l_4)
_, r2_4, _ = rf_model4.get_metrics(cv_t4, l_t4)
r2s_list.append(r2_4)

/att/gpfsfs/briskfs01/ppl/cssprad1/projects/rapids_rf_lake_depth/models/custom_RF.py:155: UserWarning: For reproducible results in Random Forest Classifier or for almost reproducible results in Random Forest Regressor, n_streams==1 is recommended. If n_streams is > 1, results may vary due to stream/thread timing differences, even when random_state is set
  verbose=param['VERBOSE'])


Scores ------
 MAE:  0.49880743
  r2:  0.7570322540773884
 MSE:  0.9274649


In [14]:
rf_model5 = rf.cuRF(hyperparameters)
cv_5, cv_t5, l_5, l_t5 = data.split(0.20)
rf_model5.train(cv_5, l_5)
_, r2_5, _ = rf_model5.get_metrics(cv_t5, l_t5)
r2s_list.append(r2_5)

/att/gpfsfs/briskfs01/ppl/cssprad1/projects/rapids_rf_lake_depth/models/custom_RF.py:155: UserWarning: For reproducible results in Random Forest Classifier or for almost reproducible results in Random Forest Regressor, n_streams==1 is recommended. If n_streams is > 1, results may vary due to stream/thread timing differences, even when random_state is set
  verbose=param['VERBOSE'])


Scores ------
 MAE:  0.48655018
  r2:  0.7688098643156092
 MSE:  0.87456846


In [15]:
rf_model6 = rf.cuRF(hyperparameters)
cv_6, cv_t6, l_6, l_t6 = data.split(0.20)
rf_model6.train(cv_6, l_6)
_, r2_6, _ = rf_model6.get_metrics(cv_t6, l_t6)
r2s_list.append(r2_6)

/att/gpfsfs/briskfs01/ppl/cssprad1/projects/rapids_rf_lake_depth/models/custom_RF.py:155: UserWarning: For reproducible results in Random Forest Classifier or for almost reproducible results in Random Forest Regressor, n_streams==1 is recommended. If n_streams is > 1, results may vary due to stream/thread timing differences, even when random_state is set
  verbose=param['VERBOSE'])


Scores ------
 MAE:  0.5018901
  r2:  0.7614600032471758
 MSE:  0.8611403


In [16]:
import numpy as np

In [17]:
print(r2s_list)

[0.7403002871086541, 0.7731209960404318, 0.7570322540773884, 0.7688098643156092, 0.7614600032471758]


In [18]:
r2_mean2 = np.mean(r2s_list)
print(r2_mean2)

0.760144680957852
